In [1]:
#import dependencies
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from datasets import load_dataset, Dataset

from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch



In [2]:
#read in the csv file
file = "dataset.csv"
df = pd.read_csv(file)

In [3]:
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Encoding The Data 

In [4]:
# Manipulate the dataframe to instead show the symptoms as features with binary values

# Extract all the symptom columns
symptom_columns = [col for col in df.columns if 'Symptom_' in col]

# Extract all unique symptoms across the dataset
unique_symptoms = pd.Series(df[symptom_columns].values.ravel()).dropna().unique()

# Create new columns for each unique symptom and initialize with 0
for symptom in unique_symptoms:
    df[symptom] = 0

# Iterate over each row and mark the presence of each symptom as 1
for index, row in df.iterrows():
    for symptom in symptom_columns:
        if pd.notna(row[symptom]):
            df.loc[index, row[symptom]] = 1

# Drop the original symptom columns
df_encoded = df.drop(columns=symptom_columns)

# Display the transformed DataFrame
df_encoded.head()

/var/folders/fg/8tsgxb6s07x1cjstms_qn8pc0000gn/T/ipykernel_21110/67963145.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[symptom] = 0
/var/folders/fg/8tsgxb6s07x1cjstms_qn8pc0000gn/T/ipykernel_21110/67963145.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[symptom] = 0
/var/folders/fg/8tsgxb6s07x1cjstms_qn8pc0000gn/T/ipykernel_21110/67963145.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all

,Disease,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,continuous_sneezing,shivering,chills,watering_from_eyes,stomach_pain,...,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
0,Fungal infection,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Fungal infection,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Fungal infection,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Fungal infection,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Fungal infection,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# Initialize the label encoder model to convert the target labels to integers
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit the label encoder model and transform the target labels
df_encoded['Disease'] = label_encoder.fit_transform(df_encoded['Disease'])

df_encoded.head()

,Disease,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,continuous_sneezing,shivering,chills,watering_from_eyes,stomach_pain,...,bladder_discomfort,foul_smell_of urine,continuous_feel_of_urine,skin_peeling,silver_like_dusting,small_dents_in_nails,inflammatory_nails,blister,red_sore_around_nose,yellow_crust_ooze
0,15,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,15,0,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,15,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,15,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
save_path = "dataset_encoded.csv"
df_encoded.to_csv(save_path, index=False)

# Transformer HuggingFace Model Training 

In [7]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

# Load your encoded dataset (replace with your actual file path)
encoded_df = pd.read_csv('dataset_encoded.csv')

# Prepare the dataset by separating features and target
X = encoded_df.drop(columns=["Disease"])
y = encoded_df["Disease"]

# Create a dataset in Hugging Face format
dataset = Dataset.from_pandas(pd.DataFrame({"text": X.apply(lambda row: " ".join(row.astype(str)), axis=1), "label": y}))
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 4920
})

In [9]:
# Load the tokenizer and model from Hugging Face's pre-trained models
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
#change the model device to cuda 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Load a pre-trained model suitable for classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(y.unique())).to(device)
# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # Normally you'd have a separate validation dataset
)

# Train the model
trainer.train()

Map:   0%|          | 0/4920 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/924 [00:00<?, ?it/s]

  0%|          | 0/615 [00:00<?, ?it/s]

{'eval_loss': 3.649989366531372, 'eval_runtime': 1243.1642, 'eval_samples_per_second': 3.958, 'eval_steps_per_second': 0.495, 'epoch': 1.0}
{'loss': 3.5719, 'grad_norm': 9.612815856933594, 'learning_rate': 9.177489177489179e-06, 'epoch': 1.62}


  0%|          | 0/615 [00:00<?, ?it/s]

{'eval_loss': 2.726978302001953, 'eval_runtime': 774.6222, 'eval_samples_per_second': 6.351, 'eval_steps_per_second': 0.794, 'epoch': 2.0}


  0%|          | 0/615 [00:00<?, ?it/s]

{'eval_loss': 2.376126527786255, 'eval_runtime': 90.8088, 'eval_samples_per_second': 54.18, 'eval_steps_per_second': 6.772, 'epoch': 3.0}
{'train_runtime': 17709.0454, 'train_samples_per_second': 0.833, 'train_steps_per_second': 0.052, 'train_loss': 3.1733357483174376, 'epoch': 3.0}


TrainOutput(global_step=924, training_loss=3.1733357483174376, metrics={'train_runtime': 17709.0454, 'train_samples_per_second': 0.833, 'train_steps_per_second': 0.052, 'total_flos': 1956578677678080.0, 'train_loss': 3.1733357483174376, 'epoch': 3.0})

In [10]:
# evaluate the model
metrics = trainer.evaluate()
print(metrics)

  0%|          | 0/615 [00:00<?, ?it/s]

{'eval_loss': 2.376126527786255, 'eval_runtime': 488.5293, 'eval_samples_per_second': 10.071, 'eval_steps_per_second': 1.259, 'epoch': 3.0}


In [11]:
# calculte the perplexity 
import math
perplexity = math.exp(metrics["eval_loss"])
print(perplexity)

10.763131325589743


In [15]:
# Save the model
model.save_pretrained("disease-prediction-transformer")
# Saving the tokenizer
tokenizer.save_pretrained('disease-prediction-transformer')


('disease-prediction-transformer/tokenizer_config.json',
 'disease-prediction-transformer/special_tokens_map.json',
 'disease-prediction-transformer/vocab.txt',
 'disease-prediction-transformer/added_tokens.json',
 'disease-prediction-transformer/tokenizer.json')